### Candes and Recht algorithm

In [ ]:
k_dict = {'50x30': {'row_order': 50,
  'col_order': 30,
  'prob': [0.1, 0.2, 0.3, 0.32, 0.34, 0.36],
  '#missing_entries': [150, 300, 450, 480, 510, 540],
  'k': [47, 49, 49, 49, 49, 49],
  'r': [30, 30, 29, 29, 29],
  'FPTtime': [0.22645235061645508,
   0.30129432678222656,
   4.639664888381958,
   5.561783075332642,
   8.93376088142395,
   14.116338968276978],
  'ConvexTime': [30.946329832077026,
   29.21294355392456,
   30.66036343574524,
   32.6079523563385,
   32.3644814491272,
   32.75907897949219]}}

`M: The true matrix.
n_1: The number of row of the matrix M.
n_2: The number of column of the matrix M.
r: The rank of the matrix M.
omega: The sample set. observed entry (i, j) in the sample set omega.
M_: Input of algorithm. The matrix to recover.
X_: The matrix to recover in SDP.
X: The solution of nuclear norm minimization.`

In [52]:
import random
import math
import itertools
import time
import random
import numpy as np
from cvxpy import *

In [ ]:
# import random
# import math
# import itertools
# import time
# import random
# import numpy as np
# from cvxpy import *

# Noting starting time
start = time.time()
random.seed(0)
rank, p = 5, 1


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def _solve(M, omega):
    n_1 = M.shape[0]
    n_2 = M.shape[1]
    X_ = Semidef(n_1 + n_2)
    objective = Minimize(trace(X_))

    constraints = [(X_ == X_.T)]  # add symmetric constraint.
    for i, j in omega:
        constr = (X_[i, j + n_1] == M[i, j])
        constraints.append(constr)
    problem = Problem(objective, constraints)
    problem.solve(solver=CVXOPT)

    print("STATUS       :", problem.status)
    print("OPTIMAL VALUE:", problem.value)

    X0 = X_.value
    # check optimizer's solution is symmetric.
    print("|X0-X0.T|_F  :", np.linalg.norm(np.subtract(X0, X0.T), "fro"))
    return X_.value[:n_1, n_1:]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def _generate_omega(n_1, n_2, m):
    random.seed(0)
    return random.sample([(i, j) for i in range(n_1-1) for j in range(n_2-1)], m)


def _get_mask_matrix(n_1, n_2, omega):
    """
    If we observed entry (i, j) of matrix M, the entry of mask matrix is 1,
    Otherwise 0.
    """
    mask = np.zeros((n_1, n_2), dtype=np.int8)
    for i, j in omega:
        mask[i, j] = 1
    return mask


def _get_abs_max_from_matrix(M):
    return np.max(np.absolute(M))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def main(n_1, n_2, r, m):
    print("#row of M    :", n_1)
    print("#column of M :", n_2)
    print("#sample      :", m)
    random.seed(0)
    L = np.array(np.random.randint(p,size=(row_order,rank)),dtype=float)
    R = np.array(np.random.randint(p,size=(col_order,rank)),dtype=float)
    M = L.dot(R.T)
    M_abs_max = _get_abs_max_from_matrix(M)
    # print("RANK of M    :", np.linalg.matrix_rank(M))
    print("|M|_*        :", np.linalg.norm(M, "nuc"))
    M_rank = np.linalg.matrix_rank(M)
    print("RANK of M    :", M_rank)
    U, S, V_T = np.linalg.svd(M, full_matrices=False)
    print(S)

    omega = _generate_omega(n_1, n_2, m)
    
    #for calculating K
    i,_ = zip(*omega)
    k_dict[f'{row_order}x{col_order}']['k'] = len(len(set(i)))
    
    mask = _get_mask_matrix(n_1, n_2, omega)

    # M_ is for training data removing the data.
    # This block should not affect the result.
    # Just for defensive programming.
    # M_ = np.random.uniform(-M_abs_max, M_abs_max, M.shape)
    M_ = M.copy()
    np.place(M_, 1 - mask, M_abs_max * M_abs_max)
    X = _solve(M_, omega)
    X_rank = np.linalg.matrix_rank(X)
    print("RANK of X    :", X_rank)
    print("|X|_*        :", np.linalg.norm(X, "nuc"))
    U, S, V_T = np.linalg.svd(X, full_matrices=False)
    print(S)

    E = np.subtract(M, X)
    E_train = E.copy()
    np.place(E_train, 1 - mask, 0)
    print('TRAIN RMSE   :', np.linalg.norm(E_train, "fro") / m)
    E_test = E.copy()
    np.place(E_test, mask, 0)
    print('TEST  RMSE   :', np.linalg.norm(E_test, "fro") / (n_1 * n_2 - m))

    frobenius_norm_ratio = (
        np.linalg.norm(X - M, "fro") / np.linalg.norm(M, "fro"))
    print("|X-M|_F/|M|_F:", frobenius_norm_ratio)  # the metric of paper.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
if __name__ == '__main__':
    row_order,col_order = (100,50)
    
    # for plotting purposes
    k_dict[f'{row_order}x{col_order}']={}
    k_dict[f'{row_order}x{col_order}']['row_order'] = row_order
    k_dict[f'{row_order}x{col_order}']['col_order'] = col_order
    k_dict[f'{row_order}x{col_order}']['r'] = r
    q = 0.4
    main(row_order,col_order, r=2, m=int(row_order * col_order * q))
    
# Noting ending time
end = time.time()
k_dict[f'{row_order}x{col_order}']['time'] = end-start

#row of M    : 100
#column of M : 50
#sample      : 2000
|M|_*        : 2133.2447766751675
RANK of M    : 5
[1.53594198e+03 1.71085592e+02 1.60756352e+02 1.55529771e+02
 1.09931082e+02 1.49045517e-13 1.49045517e-13 1.49045517e-13
 1.49045517e-13 1.49045517e-13 1.49045517e-13 1.49045517e-13
 1.49045517e-13 1.49045517e-13 1.49045517e-13 1.49045517e-13
 1.49045517e-13 1.49045517e-13 1.49045517e-13 1.49045517e-13
 1.49045517e-13 1.49045517e-13 1.49045517e-13 1.49045517e-13
 1.49045517e-13 1.49045517e-13 1.49045517e-13 1.49045517e-13
 1.49045517e-13 1.49045517e-13 1.49045517e-13 1.49045517e-13
 1.49045517e-13 1.49045517e-13 1.49045517e-13 1.49045517e-13
 1.49045517e-13 1.49045517e-13 1.49045517e-13 1.49045517e-13
 1.49045517e-13 1.49045517e-13 1.49045517e-13 1.49045517e-13
 1.49045517e-13 1.49045517e-13 1.49045517e-13 1.49045517e-13
 1.49045517e-13 4.74042001e-14]


In [ ]:
random.seed(0)
i,_ = zip(*random.sample([(i, j) for i in range(4) for j in range(4)], 5))
len(set(i))

In [49]:
random.seed(0)
random.sample([(i, j) for i in range(4) for j in range(4)],5)

[(3, 0), (3, 3), (1, 2), (0, 0), (1, 0)]